
# Task 1 - Dataset, Background, Significance, Motivation

For this project, I have selected the [soccer](https://www.kaggle.com/datasets/davidcariboo/player-scores?select=transfers.csv) dataset, which is publicly available on Kaggle. 


## Background and Significance




## Motivation


In [ ]:
# Importing data analysis libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Reading the dataset from the CSV's files 
df = pd.read_csv("./data/appearances.csv")

# Task 2 -Exploratory Data Analysis

## 1

1 conclusion

## 2


2 conclusion

## 3


3 conclusion

## 4

4 conclusion

## 5

5  conclusion

## Exploratory Data Analysis conclusion

# Task 3 – Problem Definition

# Task 4 - Data Preprocessing prior to model training

# Task 5 – Model Selection, Training, and Optimization

## Model 1: 

## Model 2:

## Model 3:

## Model 4:

## Model 5:

# Task 6 – Model Evaluation

## Model 1:

### Metric 1:

result

### Metric 2:

result

### Metric 3:

result

### Metric 4:

result

## Model 2:

## Model 3:

## Model 4:

## Model 5: